In [1]:
import pandas as pd
from tensorflow import keras

from sklearn.model_selection import train_test_split

from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs
from os import path
import pickle

In [2]:
import wandb
from wandb.keras import WandbCallback


In [3]:
DATA_DIR = "../../data/"

In [4]:
OUTPUT_TYPE = OutputType(False, 'binary')
UNIQUE_LABELS = [0, 1]
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)

In [5]:
pretrained_model_generator, input_encoder = load_pretrained_model("../../data/protein_bert/", "epoch_92400_sample_23500000.pkl")

In [5]:
#pretrained_model_generator, input_encoder = load_pretrained_model("../../data/protein_bert/", "checkpoint_2022_01_19.pkl")

In [7]:
model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)

In [30]:
training_callbacks = [
    keras.callbacks.ReduceLROnPlateau(patience = 1, factor = 0.25, min_lr = 1e-05, verbose = 1),
    keras.callbacks.EarlyStopping(patience = 2, restore_best_weights = True),
    #WandbCallback()
]

In [6]:
train_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_train_data.csv"), index_col=0)
valid_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_valid_data.csv"), index_col=0)
test_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_test_data.csv"), index_col=0)
train_data.head()

,Antibody_ID,heavy,light,Y
2073,6aod,EVQLVQSGAEVKKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLE...,DIVMTKSPSSLSASVGDRVTITCRASQGIRNDLGWYQQKPGKAPKR...,0
1517,4yny,EVQLVESGGGLVQPGRSLKLSCAASGFTFSNYGMAWVRQTPTKGLE...,EFVLTQPNSVSTNLGSTVKLSCKRSTGNIGSNYVNWYQQHEGRSPT...,1
2025,5xcv,EVQLVESGGGLVQPGRSLKLSCAASGFTFSNYGMAWVRQTPTKGLE...,QFVLTQPNSVSTNLGSTVKLSCKRSTGNIGSNYVNWYQQHEGRSPT...,1
2070,6and,EVQLVESGGGLVQPGGSLRLSCAASGYEFSRSWMNWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRSSQSIVHSVGNTFLEWYQQKPG...,1
666,2xqy,QVQLQQPGAELVKPGASVKMSCKASGYSFTSYWMNWVKQRPGRGLE...,DIVLTQSPASLALSLGQRATISCRASKSVSTSGYSYMYWYQQKPGQ...,0


In [7]:
train_data["seq"] = train_data["heavy"] + train_data["light"]
valid_data["seq"] = valid_data["heavy"] + valid_data["light"]
test_data["seq"] = test_data["heavy"] + test_data["light"]

In [23]:
wandb.config = {
  "learning_rate": 1e-04,
  "epochs": 65,
  "batch_size": 32
}

In [31]:
finetune(model_generator, input_encoder, OUTPUT_SPEC, train_data['seq'], train_data['Y'], valid_data['seq'], valid_data['Y'], \
        seq_len = 512, batch_size = 128, max_epochs_per_stage = 40, lr = 1e-04, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 1024, final_lr = 1e-05, callbacks = training_callbacks)

[2022_01_25-16:32:40] Training set: Filtered out 0 of 1338 (0.0%) records of lengths exceeding 510.
[2022_01_25-16:32:40] Validation set: Filtered out 0 of 120 (0.0%) records of lengths exceeding 510.
[2022_01_25-16:32:40] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/tf-gpu/lib/python3.7/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/40
11/11 [==============================] - 97s 8s/step - loss: 0.7128 - val_loss: 0.5163 - lr: 0.0100
Epoch 2/40
11/11 [==============================] - ETA: 0s - loss: 0.5223
Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.0024999999441206455.
11/11 [==============================] - 87s 8s/step - loss: 0.5223 - val_loss: 0.5215 - lr: 0.0100
Epoch 3/40
11/11 [==============================] - 88s 8s/step - loss: 0.4733 - val_loss: 0.4881 - lr: 0.0025
Epoch 4/40
11/11 [==============================] - 88s 8s/step - loss: 0.4678 - val_loss: 0.4809 - lr: 0.0025
Epoch 5/40
11/11 [==============================] - 87s 8s/step - loss: 0.4583 - val_loss: 0.4767 - lr: 0.0025
Epoch 6/40
11/11 [==============================] - 88s 8s/step - loss: 0.4537 - val_loss: 0.4747 - lr: 0.0025
Epoch 7/40
11/11 [==============================] - 88s 8s/step - loss: 0.4525 - val_loss: 0.4691 - lr: 0.0025
Epoch 8/40
 3/11 [=======>......................] - ETA: 1:02 - loss: 0.4423

KeyboardInterrupt: 

In [25]:
results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
        start_seq_len = 512, start_batch_size = 32)

In [26]:
print('Test-set performance:')
display(results)

print('Confusion matrix:')
display(confusion_matrix)

Test-set performance:


,# records,AUC
Model seq len,,
512,119,0.936368
All,119,0.936368


Confusion matrix:


,0,1
0,96,0
1,13,10


In [27]:
f1 = 10 / (10 + 0.5* (13 + 0))
# TN / (TN + 0.5 * (FP + FN))
f1

0.6060606060606061

In [16]:
mod = model_generator.create_model(seq_len = 512)

In [17]:
mod.save(path.join(DATA_DIR, "protein_bert/batch_32_lr_1e-4_2022_01_25.pkl"))

2022-01-25 15:36:30.188640: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../../data/protein_bert/batch_32_lr_1e-4_2022_01_25.pkl/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/batch_32_lr_1e-4_2022_01_25.pkl/assets
/home/brazdilv/.conda/envs/tf-gpu/lib/python3.7/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/home/brazdilv/.conda/envs/tf-gpu/lib/python3.7/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [8]:
def fine_tune(i, lr, epochs, batch_size):
    wandb.init(project=f"ProteinBERT_{i}", entity="kvetab")
    model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)
    training_callbacks = [
        keras.callbacks.ReduceLROnPlateau(patience = 1, factor = 0.25, min_lr = 1e-05, verbose = 1),
        keras.callbacks.EarlyStopping(patience = 2, restore_best_weights = True),
        WandbCallback()
    ]
    
    wandb.config = {
      "learning_rate": lr,
      "epochs": epochs * 2,
      "batch_size": batch_size
    }
    
    finetune(model_generator, input_encoder, OUTPUT_SPEC, train_data['seq'], train_data['Y'], valid_data['seq'], valid_data['Y'], \
        seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epochs, lr = lr, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 512, final_lr = lr / 10, callbacks = training_callbacks)
    results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
        start_seq_len = 512, start_batch_size = batch_size)
    print(f"Model number {i} trained with learning rate {lr}:")
    print('Test-set performance:')
    display(results)

    print('Confusion matrix:')
    display(confusion_matrix)
    mod = model_generator.create_model(seq_len = 512)
    mod.save(path.join(DATA_DIR, f"protein_bert/{i}_batch_{batch_size}_lr_{lr}_2022_01_25.pkl"))

In [ ]:
i = 0
batch_size = 64
epoch_num = 50
for learning_rate in [1e-5, 5e-5, 1e-4, 5e-4]:
    fine_tune(f"{i:02d}", learning_rate, epoch_num, batch_size)
    i += 1

[2022_01_26-17:23:10] Training set: Filtered out 0 of 1338 (0.0%) records of lengths exceeding 510.
[2022_01_26-17:23:11] Validation set: Filtered out 0 of 120 (0.0%) records of lengths exceeding 510.
[2022_01_26-17:23:11] Training with frozen pretrained layers...


2022-01-26 17:23:11.044361: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
/home/brazdilv/.conda/envs/tf-gpu/lib/python3.7/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/50
21/21 [==============================] - 113s 5s/step - loss: 0.7342 - val_loss: 0.6132 - lr: 0.0100
Epoch 2/50
21/21 [==============================] - 90s 4s/step - loss: 0.5042 - val_loss: 0.5167 - lr: 0.0100
Epoch 3/50
21/21 [==============================] - 90s 4s/step - loss: 0.5016 - val_loss: 0.4712 - lr: 0.0100
Epoch 4/50
21/21 [==============================] - 90s 4s/step - loss: 0.4621 - val_loss: 0.4606 - lr: 0.0100
Epoch 5/50
21/21 [==============================] - 100s 5s/step - loss: 0.4341 - val_loss: 0.4351 - lr: 0.0100
Epoch 6/50
21/21 [==============================] - 98s 5s/step - loss: 0.4177 - val_loss: 0.4287 - lr: 0.0100
Epoch 7/50
21/21 [==============================] - 92s 4s/step - loss: 0.4200 - val_loss: 0.4166 - lr: 0.0100
Epoch 8/50
21/21 [==============================] - 97s 5s/step - loss: 0.4277 - val_loss: 0.4164 - lr: 0.0100
Epoch 9/50
21/21 [==============================] - ETA: 0s - loss: 0.4395
Epoch 00009: ReduceLROnPlateau redu

,# records,AUC
Model seq len,,
512,119,0.939538
All,119,0.939538


Confusion matrix:


...

Confusion matrix:


,0,1
0,96,0
1,15,8


/home/brazdilv/.conda/envs/tf-gpu/lib/python3.7/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


INFO:tensorflow:Assets written to: ../../data/protein_bert/03_batch_64_lr_0.0005_2022_01_25.pkl/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/03_batch_64_lr_0.0005_2022_01_25.pkl/assets
/home/brazdilv/.conda/envs/tf-gpu/lib/python3.7/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/home/brazdilv/.conda/envs/tf-gpu/lib/python3.7/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [12]:
f1 = 8 / (8 + 0.5* (15 + 0))
# TN / (TN + 0.5 * (FP + FN))
f1

0.5161290322580645

In [9]:
i = 4
epoch_num = 60
batch_size = 64
learning_rate = 1e-5
fine_tune(f"{i:02d}", learning_rate, epoch_num, batch_size)

wandb: Currently logged in as: kvetab (use `wandb login --relogin` to force relogin)
/home/brazdilv/.conda/envs/tf-gpu/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


[2022_01_27-10:06:31] Training set: Filtered out 0 of 1338 (0.0%) records of lengths exceeding 510.
[2022_01_27-10:06:31] Validation set: Filtered out 0 of 120 (0.0%) records of lengths exceeding 510.
[2022_01_27-10:06:31] Training with frozen pretrained layers...


2022-01-27 10:06:31.462191: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
/home/brazdilv/.conda/envs/tf-gpu/lib/python3.7/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/60
21/21 [==============================] - ETA: 0s - loss: 0.7227

/home/brazdilv/.conda/envs/tf-gpu/lib/python3.7/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
wandb: ERROR Can't save model, h5py returned error: 
wandb: ERROR Layer GlobalAttention has arguments ['self', 'n_heads', 'd_key', 'd_value']
wandb: ERROR in `__init__` and therefore must override `get_config()`.
wandb: ERROR 
wandb: ERROR Example:
wandb: ERROR 
wandb: ERROR class CustomLayer(keras.layers.Layer):
wandb: ERROR     def __init__(self, arg1, arg2):
wandb: ERROR         super().__init__()
wandb: ERROR         self.arg1 = arg1
wandb: ERROR         self.arg2 = arg2
wandb: ERROR 
wandb: ERROR     def get_config(self):
wandb: ERROR         config = super().get_config()
wandb: ERROR         config.update({
wandb: ERROR             "arg1": self.arg1,
wandb: ERROR             "arg

...

Confusion matrix:


,0,1
0,96,0
1,15,8


/home/brazdilv/.conda/envs/tf-gpu/lib/python3.7/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2022-01-27 11:58:35.240804: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../../data/protein_bert/04_batch_64_lr_1e-05_2022_01_25.pkl/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/04_batch_64_lr_1e-05_2022_01_25.pkl/assets
/home/brazdilv/.conda/envs/tf-gpu/lib/python3.7/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/home/brazdilv/.conda/envs/tf-gpu/lib/python3.7/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)
